# Appendix: Categorical data

[Vir](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categorical-memory)

In [2]:
import pandas as pd 
import numpy as np

This is an introduction to pandas categorical data type, including a short comparison with R’s factor.

Categoricals are a pandas data type corresponding to categorical variables in statistics. A categorical variable takes on a limited, and usually fixed, number of possible values (categories; levels in R). Examples are gender, social class, blood type, country affiliation, observation time or rating via Likert scales.

In contrast to statistical categorical variables, categorical data might have an order (e.g. ‘strongly agree’ vs ‘agree’ or ‘first observation’ vs. ‘second observation’), but numerical operations (additions, divisions, …) are not possible.

All values of categorical data are either in categories or np.nan. Order is defined by the order of categories, not lexical order of the values. Internally, the data structure consists of a categories array and an integer array of codes which point to the real value in the categories array.

The categorical data type is useful in the following cases:

- A string variable consisting of only a few different values. Converting such a string variable to a categorical variable will save some memory, see here.
- The lexical order of a variable is not the same as the logical order (“one”, “two”, “three”). By converting to a categorical and specifying an order on the categories, sorting and min/max will use the logical order instead of the lexical order, see here.
- As a signal to other Python libraries that this column should be treated as a categorical variable (e.g. to use suitable statistical methods or plot types).
See also the API docs on categoricals.

## Object creation

### Series creation

Categorical Series or columns in a DataFrame can be created in several ways:

By specifying dtype="category" when constructing a Series:

In [3]:
 s = pd.Series(["a", "b", "c", "a"], dtype="category")

In [4]:
s

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

By converting an existing Series or column to a category dtype:

In [5]:
df = pd.DataFrame({"A": ["a", "b", "c", "a"]})

In [6]:
df["B"] = df["A"].astype('category')

In [7]:
df

,A,B
0,a,a
1,b,b
2,c,c
3,a,a


By using special functions, such as cut(), which groups data into discrete bins. See the example on tiling in the docs.

In [8]:
 df = pd.DataFrame({'value': np.random.randint(0, 100, 20)})

In [9]:
labels = ["{0} - {1}".format(i, i + 9) for i in range(0, 100, 10)]

In [10]:
labels

['0 - 9',
 '10 - 19',
 '20 - 29',
 '30 - 39',
 '40 - 49',
 '50 - 59',
 '60 - 69',
 '70 - 79',
 '80 - 89',
 '90 - 99']

In [11]:
df['group'] = pd.cut(df.value, range(0, 105, 10), right=False, labels=labels)

In [12]:
df.head(10)

,value,group
0,10,10 - 19
1,4,0 - 9
2,54,50 - 59
3,67,60 - 69
4,70,70 - 79
5,83,80 - 89
6,25,20 - 29
7,12,10 - 19
8,81,80 - 89
9,31,30 - 39


By passing a pandas.Categorical object to a Series or assigning it to a DataFrame.

In [13]:
raw_cat = pd.Categorical(["a", "b", "c", "a"], 
                         categories=["b", "c", "d"],
                         ordered=False)

In [16]:
s = pd.Series(raw_cat)

In [17]:
s

0    NaN
1      b
2      c
3    NaN
dtype: category
Categories (3, object): [b, c, d]

In [18]:
df = pd.DataFrame({"A": ["a", "b", "c", "a"]})

In [19]:
df["B"] = raw_cat

In [20]:
df

,A,B
0,a,NaN
1,b,b
2,c,c
3,a,NaN


Categorical data has a specific category dtype:

In [21]:
df.dtypes

A      object
B    category
dtype: object

### DataFrame creation

Similar to the previous section where a single column was converted to categorical, all columns in a DataFrame can be batch converted to categorical either during or after construction.

This can be done during construction by specifying dtype="category" in the DataFrame constructor:

In [22]:
df = pd.DataFrame({'A': list('abca'), 'B': list('bccd')}, dtype="category")

In [23]:
df.dtypes

A    category
B    category
dtype: object

Note that the categories present in each column differ; the conversion is done column by column, so only labels present in a given column are categories:

In [24]:
df['A']

0    a
1    b
2    c
3    a
Name: A, dtype: category
Categories (3, object): [a, b, c]

In [25]:
df['B']

0    b
1    c
2    c
3    d
Name: B, dtype: category
Categories (3, object): [b, c, d]

> New in version 0.23.0.

Analogously, all columns in an existing DataFrame can be batch converted using DataFrame.astype():

In [26]:
df = pd.DataFrame({'A': list('abca'), 'B': list('bccd')})

In [27]:
df_cat = df.astype('category')

In [28]:
df_cat.dtypes

A    category
B    category
dtype: object

This conversion is likewise done column by column:

In [29]:
df_cat['A']

0    a
1    b
2    c
3    a
Name: A, dtype: category
Categories (3, object): [a, b, c]

In [30]:
df_cat['B']

0    b
1    c
2    c
3    d
Name: B, dtype: category
Categories (3, object): [b, c, d]

### Controlling behavior

In the examples above where we passed dtype='category', we used the default behavior:
- Categories are inferred from the data.
- Categories are unordered.

To control those behaviors, instead of passing 'category', use an instance of CategoricalDtype.

In [31]:
from pandas.api.types import CategoricalDtype

In [ ]:
To control those behaviors, instead of passing 'category', use an instance of CategoricalDtype.

In [32]:
s = pd.Series(["a", "b", "c", "a"])

In [33]:
cat_type = CategoricalDtype(categories=["b", "c", "d"], ordered=True)

In [34]:
s_cat = s.astype(cat_type)

In [35]:
s_cat

0    NaN
1      b
2      c
3    NaN
dtype: category
Categories (3, object): [b < c < d]

Similarly, a CategoricalDtype can be used with a DataFrame to ensure that categories are consistent among all columns.

In [36]:
from pandas.api.types import CategoricalDtype

In [37]:
df = pd.DataFrame({'A': list('abca'), 'B': list('bccd')})

In [38]:
cat_type = CategoricalDtype(categories=list('abcd'), ordered=True)

In [39]:
df_cat = df.astype(cat_type)

In [40]:
df_cat['A']

0    a
1    b
2    c
3    a
Name: A, dtype: category
Categories (4, object): [a < b < c < d]

In [41]:
df_cat['B']

0    b
1    c
2    c
3    d
Name: B, dtype: category
Categories (4, object): [a < b < c < d]

> Note: To perform table-wise conversion, where all labels in the entire DataFrame are used as categories for each column, the categories parameter can be determined programmatically by categories = pd.unique(df.to_numpy().ravel()).

If you already have codes and categories, you can use the from_codes() constructor to save the factorize step during normal constructor mode:

In [42]:
splitter = np.random.choice([0, 1], 5, p=[0.5, 0.5])

In [43]:
s = pd.Series(pd.Categorical.from_codes(splitter, categories=["train", "test"]))

### Regaining original data

To get back to the original Series or NumPy array, use Series.astype(original_dtype) or np.asarray(categorical)

In [44]:
 s = pd.Series(["a", "b", "c", "a"])

In [45]:
s

0    a
1    b
2    c
3    a
dtype: object

In [46]:
s2 = s.astype('category')

In [47]:
s2

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

In [48]:
s2.astype(str)

0    a
1    b
2    c
3    a
dtype: object

In [49]:
np.asarray(s2)

array(['a', 'b', 'c', 'a'], dtype=object)

## CategoricalDtype

Changed in version 0.21.0.

A categorical’s type is fully described by
- categories: a sequence of unique values and no missing values
- ordered: a boolean

This information can be stored in a CategoricalDtype. The categories argument is optional, which implies that the actual categories should be inferred from whatever is present in the data when the pandas.Categorical is created. The categories are assumed to be unordered by default.

In [50]:
CategoricalDtype(['a', 'b', 'c'])

CategoricalDtype(categories=['a', 'b', 'c'], ordered=None)

In [51]:
CategoricalDtype(['a', 'b', 'c'], ordered=True)

CategoricalDtype(categories=['a', 'b', 'c'], ordered=True)

In [52]:
CategoricalDtype()

CategoricalDtype(categories=None, ordered=None)

A CategoricalDtype can be used in any place pandas expects a dtype. For example pandas.read_csv(), pandas.DataFrame.astype(), or in the Series constructor.

> Note: As a convenience, you can use the string 'category' in place of a CategoricalDtype when you want the default behavior of the categories being unordered, and equal to the set values present in the array. In other words, dtype='category' is equivalent to dtype=CategoricalDtype().

### Equality semantics

Two instances of CategoricalDtype compare equal whenever they have the same categories and order. When comparing two unordered categoricals, the order of the categories is not considered.

In [53]:
c1 = CategoricalDtype(['a', 'b', 'c'], ordered=False)

In [54]:
# Equal, since order is not considered when ordered=False
c1 == CategoricalDtype(['b', 'c', 'a'], ordered=False)

True

In [55]:
# Unequal, since the second CategoricalDtype is ordered
c1 == CategoricalDtype(['a', 'b', 'c'], ordered=True)

False

All instances of CategoricalDtype compare equal to the string 'category'.

In [56]:
c1 == 'category'

True

> Warning: Since dtype='category' is essentially CategoricalDtype(None, False), and since all instances CategoricalDtype compare equal to 'category', all instances of CategoricalDtype compare equal to a CategoricalDtype(None, False), regardless of categories or ordered.

## Description

Using describe() on categorical data will produce similar output to a Series or DataFrame of type string.

In [57]:
cat = pd.Categorical(["a", "c", "c", np.nan], categories=["b", "a", "c"])

In [58]:
df = pd.DataFrame({"cat": cat, "s": ["a", "c", "c", np.nan]})

In [59]:
df.describe()

,cat,s
count,3,3
unique,2,2
top,c,c
freq,2,2


In [60]:
df["cat"].describe()

count     3
unique    2
top       c
freq      2
Name: cat, dtype: object

## Working with categories

Categorical data has a categories and a ordered property, which list their possible values and whether the ordering matters or not. These properties are exposed as s.cat.categories and s.cat.ordered. If you don’t manually specify categories and ordering, they are inferred from the passed arguments.

In [61]:
s = pd.Series(["a", "b", "c", "a"], dtype="category")

In [64]:
s

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

In [62]:
s.cat.categories

Index(['a', 'b', 'c'], dtype='object')

In [63]:
s.cat.ordered

False

It’s also possible to pass in the categories in a specific order:

In [65]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a"], categories=["c", "b", "a"]))

In [66]:
s.cat.categories

Index(['c', 'b', 'a'], dtype='object')

In [67]:
s.cat.ordered

False

> Note: New categorical data are not automatically ordered. You must explicitly pass ordered=True to indicate an ordered Categorical.

> Note: The result of unique() is not always the same as Series.cat.categories, because Series.unique() has a couple of guarantees, namely that it returns categories in the order of appearance, and it only includes values that are actually present.

In [68]:
s = pd.Series(list('babc')).astype(CategoricalDtype(list('abcd')))

In [69]:
s

0    b
1    a
2    b
3    c
dtype: category
Categories (4, object): [a, b, c, d]

In [70]:
s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [71]:
s.unique()

[b, a, c]
Categories (3, object): [b, a, c]

### Renaming categories

Renaming categories is done by assigning new values to the Series.cat.categories property or by using the rename_categories() method:

In [72]:
s = pd.Series(["a", "b", "c", "a"], dtype="category")

In [73]:
s

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

In [74]:
s.cat.categories = ["Group %s" % g for g in s.cat.categories]

In [75]:
s

0    Group a
1    Group b
2    Group c
3    Group a
dtype: category
Categories (3, object): [Group a, Group b, Group c]

In [76]:
s = s.cat.rename_categories([1, 2, 3])

In [77]:
s

0    1
1    2
2    3
3    1
dtype: category
Categories (3, int64): [1, 2, 3]

In [78]:
s = s.cat.rename_categories({1: 'x', 2: 'y', 3: 'z'})

In [79]:
s

0    x
1    y
2    z
3    x
dtype: category
Categories (3, object): [x, y, z]

> Note: Be aware that assigning new categories is an inplace operation, while most other operations under Series.cat per default return a new Series of dtype category.

Categories must be unique or a ValueError is raised:

In [80]:
try:
    s.cat.categories = [1, 1, 1]
except ValueError as e:
    print("ValueError:", str(e))

ValueError: Categorical categories must be unique


Categories must also not be NaN or a ValueError is raised:

In [81]:
try:
    s.cat.categories = [1, 2, np.nan]
except ValueError as e:
    print("ValueError:", str(e))

ValueError: Categorial categories cannot be null


### Appending new categories

Appending categories can be done by using the add_categories() method:

In [82]:
s = s.cat.add_categories([4])

In [83]:
s

0    x
1    y
2    z
3    x
dtype: category
Categories (4, object): [x, y, z, 4]

In [84]:
s.cat.categories

Index(['x', 'y', 'z', 4], dtype='object')

### Removing categories

Removing categories can be done by using the remove_categories() method. Values which are removed are replaced by np.nan.:

In [85]:
s = s.cat.remove_categories([4])

In [86]:
s

0    x
1    y
2    z
3    x
dtype: category
Categories (3, object): [x, y, z]

### Removing unused categories

Removing unused categories can also be done:

In [87]:
s = pd.Series(pd.Categorical(["a", "b", "a"], categories=["a", "b", "c", "d"]))

In [88]:
s

0    a
1    b
2    a
dtype: category
Categories (4, object): [a, b, c, d]

In [89]:
s.cat.remove_unused_categories()

0    a
1    b
2    a
dtype: category
Categories (2, object): [a, b]

### Setting categories

If you want to do remove and add new categories in one step (which has some speed advantage), or simply set the categories to a predefined scale, use set_categories().

In [90]:
s = pd.Series(["one", "two", "four", "-"], dtype="category")

In [91]:
s

0     one
1     two
2    four
3       -
dtype: category
Categories (4, object): [-, four, one, two]

In [92]:
s = s.cat.set_categories(["one", "two", "three", "four"])

In [93]:
s

0     one
1     two
2    four
3     NaN
dtype: category
Categories (4, object): [one, two, three, four]

> Note: Be aware that Categorical.set_categories() cannot know whether some category is omitted intentionally or because it is misspelled or (under Python3) due to a type difference (e.g., NumPy S1 dtype and Python strings). This can result in surprising behaviour!

## Sorting and order

If categorical data is ordered (s.cat.ordered == True), then the order of the categories has a meaning and certain operations are possible. If the categorical is unordered, .min()/.max() will raise a TypeError.

In [94]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a"], ordered=False))

In [96]:
s.sort_values(inplace=True)

In [97]:
s = pd.Series(["a", "b", "c", "a"]).astype(CategoricalDtype(ordered=True))

In [98]:
s.sort_values(inplace=True)

In [99]:
s

0    a
3    a
1    b
2    c
dtype: category
Categories (3, object): [a < b < c]

In [100]:
s.min(), s.max()

('a', 'c')

You can set categorical data to be ordered by using as_ordered() or unordered by using as_unordered(). These will by default return a new object.

In [101]:
s.cat.as_ordered()

0    a
3    a
1    b
2    c
dtype: category
Categories (3, object): [a < b < c]

In [102]:
s.cat.as_unordered()

0    a
3    a
1    b
2    c
dtype: category
Categories (3, object): [a, b, c]

Sorting will use the order defined by categories, not any lexical order present on the data type. This is even true for strings and numeric data:

In [103]:
s = pd.Series([1, 2, 3, 1], dtype="category")

In [104]:
s = s.cat.set_categories([2, 3, 1], ordered=True)

In [105]:
s

0    1
1    2
2    3
3    1
dtype: category
Categories (3, int64): [2 < 3 < 1]

In [108]:
s.sort_values(inplace=True)

In [109]:
s

1    2
2    3
0    1
3    1
dtype: category
Categories (3, int64): [2 < 3 < 1]

In [110]:
s.min(), s.max()

(2, 1)

### Reordering

Reordering the categories is possible via the Categorical.reorder_categories() and the Categorical.set_categories() methods. For Categorical.reorder_categories(), all old categories must be included in the new categories and no new categories are allowed. This will necessarily make the sort order the same as the categories order.

In [111]:
s = pd.Series([1, 2, 3, 1], dtype="category")

In [112]:
s = s.cat.reorder_categories([2, 3, 1], ordered=True)

In [113]:
s

0    1
1    2
2    3
3    1
dtype: category
Categories (3, int64): [2 < 3 < 1]

In [114]:
s.sort_values(inplace=True)

In [115]:
s

1    2
2    3
0    1
3    1
dtype: category
Categories (3, int64): [2 < 3 < 1]

In [116]:
s.min(), s.max()

(2, 1)

> Note Note the difference between assigning new categories and reordering the categories: the first renames categories and therefore the individual values in the Series, but if the first position was sorted last, the renamed value will still be sorted last. Reordering means that the way values are sorted is different afterwards, but not that individual values in the Series are changed.

> Note: If the Categorical is not ordered, Series.min() and Series.max() will raise TypeError. Numeric operations like +, -, *, / and operations based on them (e.g. Series.median(), which would need to compute the mean between two values if the length of an array is even) do not work and raise a TypeError.

### Multi column sorting

A categorical dtyped column will participate in a multi-column sort in a similar manner to other columns. The ordering of the categorical is determined by the categories of that column.

In [117]:
dfs = pd.DataFrame({'A': pd.Categorical(list('bbeebbaa'),
    categories=['e', 'a', 'b'],
    ordered=True), 'B': [1, 2, 1, 2, 2, 1, 2, 1]})

In [119]:
dfs

,A,B
0,b,1
1,b,2
2,e,1
3,e,2
4,b,2
5,b,1
6,a,2
7,a,1


In [118]:
dfs.sort_values(by=['A', 'B'])

,A,B
2,e,1
3,e,2
7,a,1
6,a,2
0,b,1
5,b,1
1,b,2
4,b,2


Reordering the categories changes a future sort.

In [120]:
dfs['A'] = dfs['A'].cat.reorder_categories(['a', 'b', 'e'])

In [121]:
dfs.sort_values(by=['A', 'B'])

,A,B
7,a,1
6,a,2
0,b,1
5,b,1
1,b,2
4,b,2
2,e,1
3,e,2


## Comparisons

Comparing categorical data with other objects is possible in three cases:
- Comparing equality (== and !=) to a list-like object (list, Series, array, …) of the same length as the categorical data.
- All comparisons (==, !=, >, >=, <, and <=) of categorical data to another categorical Series, when ordered==True and the categories are the same.
- All comparisons of a categorical data to a scalar.

All other comparisons, especially “non-equality” comparisons of two categoricals with different categories or a categorical with any list-like object, will raise a TypeError.

> Note:  Any “non-equality” comparisons of categorical data with a Series, np.array, list or categorical data with different categories or ordering will raise a TypeError because custom categories ordering could be interpreted in two ways: one with taking into account the ordering and one without.

In [122]:
cat = pd.Series([1, 2, 3]).astype(CategoricalDtype([3, 2, 1], ordered=True))

In [124]:
cat_base = pd.Series([2, 2, 2]).astype(CategoricalDtype([3, 2, 1], ordered=True))

In [125]:
cat_base2 = pd.Series([2, 2, 2]).astype(CategoricalDtype(ordered=True))

In [126]:
cat

0    1
1    2
2    3
dtype: category
Categories (3, int64): [3 < 2 < 1]

In [127]:
cat_base

0    2
1    2
2    2
dtype: category
Categories (3, int64): [3 < 2 < 1]

In [128]:
cat_base2

0    2
1    2
2    2
dtype: category
Categories (1, int64): [2]

Comparing to a categorical with the same categories and ordering or to a scalar works:

In [129]:
cat > cat_base

0     True
1    False
2    False
dtype: bool

In [130]:
cat > 2

0     True
1    False
2    False
dtype: bool

Equality comparisons work with any list-like object of same length and scalars:

In [131]:
cat == cat_base

0    False
1     True
2    False
dtype: bool

In [132]:
cat == np.array([1, 2, 3])

0    True
1    True
2    True
dtype: bool

In [133]:
cat == 2

0    False
1     True
2    False
dtype: bool

This doesn’t work because the categories are not the same:

In [134]:
try:
    cat > cat_base2
except TypeError as e:
    print("TypeError:", str(e))

TypeError: Categoricals can only be compared if 'categories' are the same. Categories are different lengths


If you want to do a “non-equality” comparison of a categorical series with a list-like object which is not categorical data, you need to be explicit and convert the categorical data back to the original values:

In [135]:
base = np.array([1, 2, 3])

In [136]:
try:
    cat > base
except TypeError as e:
    print("TypeError:", str(e))

TypeError: Cannot compare a Categorical for op __gt__ with type <class 'numpy.ndarray'>.
If you want to compare values, use 'np.asarray(cat) <op> other'.


In [137]:
np.asarray(cat) > base

array([False, False, False])

When you compare two unordered categoricals with the same categories, the order is not considered:

In [138]:
c1 = pd.Categorical(['a', 'b'], categories=['a', 'b'], ordered=False)

In [139]:
c2 = pd.Categorical(['a', 'b'], categories=['b', 'a'], ordered=False)

In [140]:
c1 == c2

array([ True,  True])

## Operations

Apart from Series.min(), Series.max() and Series.mode(), the following operations are possible with categorical data:

Series methods like Series.value_counts() will use all categories, even if some categories are not present in the data:

In [141]:
s = pd.Series(pd.Categorical(["a", "b", "c", "c"], categories=["c", "a", "b", "d"]))

In [142]:
s.value_counts()

c    2
b    1
a    1
d    0
dtype: int64

Groupby will also show “unused” categories:

In [143]:
cats = pd.Categorical(["a", "b", "b", "b", "c", "c", "c"], categories=["a", "b", "c", "d"])

In [144]:
df = pd.DataFrame({"cats": cats, "values": [1, 2, 2, 2, 3, 4, 5]})

In [145]:
df.groupby("cats").mean()

,values
cats,
a,1.0
b,2.0
c,4.0
d,NaN


In [146]:
cats2 = pd.Categorical(["a", "a", "b", "b"], categories=["a", "b", "c"])

In [147]:
df2 = pd.DataFrame({"cats": cats2,
    "B": ["c", "d", "c", "d"],
    "values": [1, 2, 3, 4]})

In [148]:
df2.groupby(["cats", "B"]).mean()

values
cats B        
a    c     1.0
     d     2.0
b    c     3.0
     d     4.0
c    c     NaN
     d     NaN

Pivot tables:

In [149]:
raw_cat = pd.Categorical(["a", "a", "b", "b"], categories=["a", "b", "c"])

In [150]:
df = pd.DataFrame({"A": raw_cat,
    "B": ["c", "d", "c", "d"],
    "values": [1, 2, 3, 4]})

In [151]:
pd.pivot_table(df, values='values', index=['A', 'B'])

values
A B        
a c       1
  d       2
b c       3
  d       4

## Data munging

The optimized pandas data access methods .loc, .iloc, .at, and .iat, work as normal. The only difference is the return type (for getting) and that only values already in categories can be assigned.

### Getting

If the slicing operation returns either a DataFrame or a column of type Series, the category dtype is preserved.

In [152]:
idx = pd.Index(["h", "i", "j", "k", "l", "m", "n"])

In [153]:
cats = pd.Series(["a", "b", "b", "b", "c", "c", "c"], dtype="category", index=idx)

In [154]:
values = [1, 2, 2, 2, 3, 4, 5]

In [155]:
df = pd.DataFrame({"cats": cats, "values": values}, index=idx)

In [156]:
df

,cats,values
h,a,1
i,b,2
j,b,2
k,b,2
l,c,3
m,c,4
n,c,5


In [157]:
df.iloc[2:4, :]

,cats,values
j,b,2
k,b,2


In [158]:
df.iloc[2:4, :].dtypes

cats      category
values       int64
dtype: object

In [159]:
df.loc["h":"j", "cats"]

h    a
i    b
j    b
Name: cats, dtype: category
Categories (3, object): [a, b, c]

In [160]:
df[df["cats"] == "b"]

,cats,values
i,b,2
j,b,2
k,b,2


In [162]:
df.dtypes

cats      category
values       int64
dtype: object

An example where the category type is not preserved is if you take one single row: the resulting Series is of dtype object:

In [163]:
df.loc["h", :]

cats      a
values    1
Name: h, dtype: object

Returning a single item from categorical data will also return the value, not a categorical of length “1”.

In [164]:
 df.iat[0, 0]

'a'

In [166]:
df["cats"].cat.categories = ["x", "y", "z"]

In [167]:
df.at["h", "cats"]  # returns a string

'x'

To get a single value Series of type category, you pass in a list with a single value:

In [168]:
df.loc[["h"], "cats"]

h    x
Name: cats, dtype: category
Categories (3, object): [x, y, z]

### String and datetime accessors

The accessors .dt and .str will work if the s.cat.categories are of an appropriate type:

In [170]:
str_s = pd.Series(list('aabb'))

In [171]:
str_cat = str_s.astype('category')

In [172]:
str_cat

0    a
1    a
2    b
3    b
dtype: category
Categories (2, object): [a, b]

In [173]:
str_cat.str.contains("a")

0     True
1     True
2    False
3    False
dtype: bool

In [174]:
date_s = pd.Series(pd.date_range('1/1/2015', periods=5))

In [175]:
date_cat = date_s.astype('category')

In [176]:
date_cat

0   2015-01-01
1   2015-01-02
2   2015-01-03
3   2015-01-04
4   2015-01-05
dtype: category
Categories (5, datetime64[ns]): [2015-01-01, 2015-01-02, 2015-01-03, 2015-01-04, 2015-01-05]

In [177]:
date_cat.dt.day

0    1
1    2
2    3
3    4
4    5
dtype: int64

> Note: The returned Series (or DataFrame) is of the same type as if you used the .str.<method> / .dt.<method> on a Series of that type (and not of type category!).

That means, that the returned values from methods and properties on the accessors of a Series and the returned values from methods and properties on the accessors of this Series transformed to one of type category will be equal:

In [181]:
ret_s = str_s.str.contains("a")

In [182]:
ret_s

0     True
1     True
2    False
3    False
dtype: bool

In [186]:
ret_cat = str_cat.str.contains("a")

In [187]:
ret_cat

0     True
1     True
2    False
3    False
dtype: bool

In [188]:
ret_s.dtype == ret_cat.dtype

True

In [189]:
ret_s == ret_cat

0    True
1    True
2    True
3    True
dtype: bool

> Note: The work is done on the categories and then a new Series is constructed. This has some performance implication if you have a Series of type string, where lots of elements are repeated (i.e. the number of unique elements in the Series is a lot smaller than the length of the Series). In this case it can be faster to convert the original Series to one of type category and use .str.<method> or .dt.<property> on that.

### Setting

Setting values in a categorical column (or Series) works as long as the value is included in the categories:

In [190]:
idx = pd.Index(["h", "i", "j", "k", "l", "m", "n"])

In [191]:
cats = pd.Categorical(["a", "a", "a", "a", "a", "a", "a"],categories=["a", "b"])

In [192]:
values = [1, 1, 1, 1, 1, 1, 1]

In [193]:
df = pd.DataFrame({"cats": cats, "values": values}, index=idx)

In [197]:
df

,cats,values
h,a,1
i,a,1
j,b,2
k,b,2
l,a,1
m,a,1
n,a,1


In [195]:
df.iloc[2:4, :] = [["b", 2], ["b", 2]]

In [198]:
df

,cats,values
h,a,1
i,a,1
j,b,2
k,b,2
l,a,1
m,a,1
n,a,1


In [199]:
try:
    df.iloc[2:4, :] = [["c", 3], ["c", 3]]
except ValueError as e:
    print("ValueError:", str(e))

ValueError: Cannot setitem on a Categorical with a new category, set the categories first


Setting values by assigning categorical data will also check that the categories match:

In [200]:
df.loc["j":"k", "cats"] = pd.Categorical(["a", "a"], categories=["a", "b"])

In [201]:
df

,cats,values
h,a,1
i,a,1
j,a,2
k,a,2
l,a,1
m,a,1
n,a,1


In [202]:
try:
    df.loc["j":"k", "cats"] = pd.Categorical(["b", "b"], categories=["a", "b", "c"])
except ValueError as e:
    print("ValueError:", str(e))

ValueError: Cannot set a Categorical with another, without identical categories


Assigning a Categorical to parts of a column of other types will use the values:

In [203]:
df = pd.DataFrame({"a": [1, 1, 1, 1, 1], "b": ["a", "a", "a", "a", "a"]})

In [205]:
df.loc[1:2, "a"] = pd.Categorical(["b", "b"], categories=["a", "b"])

In [206]:
df.loc[2:3, "b"] = pd.Categorical(["b", "b"], categories=["a", "b"])

In [207]:
df

,a,b
0,1,a
1,b,a
2,b,b
3,1,b
4,1,a


In [208]:
df.dtypes

a    object
b    object
dtype: object

### Merging

You can concat two DataFrames containing categorical data together, but the categories of these categoricals need to be the same:

In [209]:
cat = pd.Series(["a", "b"], dtype="category")

In [211]:
vals = [1, 2]

In [212]:
df = pd.DataFrame({"cats": cat, "vals": vals})

In [213]:
res = pd.concat([df, df])

In [214]:
res

,cats,vals
0,a,1
1,b,2
0,a,1
1,b,2


In [215]:
res.dtypes

cats    category
vals       int64
dtype: object

In this case the categories are not the same, and therefore an error is raised:

In [218]:
 df_different = df.copy()

In [219]:
df_different["cats"].cat.categories = ["c", "d"]

In [221]:
try:
    pd.concat([df, df_different])
except ValueError as e:
    print("ValueError:", str(e))

The same applies to df.append(df_different).

See also the section on merge dtypes for notes about preserving merge dtypes and performance.

### Unioning

New in version 0.19.0.

If you want to combine categoricals that do not necessarily have the same categories, the union_categoricals() function will combine a list-like of categoricals. The new categories will be the union of the categories being combined.

In [222]:
from pandas.api.types import union_categoricals

In [223]:
a = pd.Categorical(["b", "c"])

In [224]:
b = pd.Categorical(["a", "b"])

In [225]:
union_categoricals([a, b])

[b, c, a, b]
Categories (3, object): [b, c, a]

By default, the resulting categories will be ordered as they appear in the data. If you want the categories to be lexsorted, use sort_categories=True argument.

In [226]:
union_categoricals([a, b], sort_categories=True)

[b, c, a, b]
Categories (3, object): [a, b, c]

union_categoricals also works with the “easy” case of combining two categoricals of the same categories and order information (e.g. what you could also append for).

In [227]:
a = pd.Categorical(["a", "b"], ordered=True)

In [228]:
b = pd.Categorical(["a", "b", "a"], ordered=True)

In [229]:
union_categoricals([a, b])

[a, b, a, b, a]
Categories (2, object): [a < b]

The below raises TypeError because the categories are ordered and not identical.

In [230]:
a = pd.Categorical(["a", "b"], ordered=True)

In [231]:
b = pd.Categorical(["a", "b", "c"], ordered=True)

In [232]:
union_categoricals([a, b])

TypeError: to union ordered Categoricals, all categories must be the same

New in version 0.20.0.

Ordered categoricals with different categories or orderings can be combined by using the ignore_ordered=True argument.

In [233]:
a = pd.Categorical(["a", "b", "c"], ordered=True)

In [234]:
b = pd.Categorical(["c", "b", "a"], ordered=True)

In [235]:
union_categoricals([a, b], ignore_order=True)

[a, b, c, c, b, a]
Categories (3, object): [a, b, c]

union_categoricals() also works with a CategoricalIndex, or Series containing categorical data, but note that the resulting array will always be a plain Categorical:

In [236]:
a = pd.Series(["b", "c"], dtype='category')

In [237]:
b = pd.Series(["a", "b"], dtype='category')

In [238]:
union_categoricals([a, b])

[b, c, a, b]
Categories (3, object): [b, c, a]

>Note: union_categoricals may recode the integer codes for categories when combining categoricals. This is likely what you want, but if you are relying on the exact numbering of the categories, be aware.

In [239]:
c1 = pd.Categorical(["b", "c"])

In [240]:
c2 = pd.Categorical(["a", "b"])

In [242]:
c1

[b, c]
Categories (2, object): [b, c]

In [243]:
c2

[a, b]
Categories (2, object): [a, b]

In [244]:
c2.codes

array([0, 1], dtype=int8)

In [245]:
c = union_categoricals([c1, c2])

In [246]:
c

[b, c, a, b]
Categories (3, object): [b, c, a]

In [247]:
# "b" is coded to 0 throughout, same as c1, different from c2
c.codes

array([0, 1, 2, 0], dtype=int8)

### Concatenation

This section describes concatenations specific to category dtype. See Concatenating objects for general description.

By default, Series or DataFrame concatenation which contains the same categories results in category dtype, otherwise results in object dtype. Use .astype or union_categoricals to get category result.

In [248]:
s1 = pd.Series(['a', 'b'], dtype='category')

In [249]:
s2 = pd.Series(['a', 'b', 'a'], dtype='category')

In [250]:
pd.concat([s1, s2])

0    a
1    b
0    a
1    b
2    a
dtype: category
Categories (2, object): [a, b]

In [251]:
s3 = pd.Series(['b', 'c'], dtype='category')

In [252]:
pd.concat([s1, s3])

0    a
1    b
0    b
1    c
dtype: object

In [253]:
pd.concat([s1, s3]).astype('category')

0    a
1    b
0    b
1    c
dtype: category
Categories (3, object): [a, b, c]

In [254]:
union_categoricals([s1.array, s3.array])

AttributeError: 'Series' object has no attribute 'array'

## Getting data in/out

You can write data that contains category dtypes to a HDFStore. See here for an example and caveats.

It is also possible to write data to and reading data from Stata format files. See here for an example and caveats.

Writing to a CSV file will convert the data, effectively removing any information about the categorical (categories and ordering). So if you read back the CSV file you have to convert the relevant columns back to category and assign the right categories and categories ordering.

In [255]:
import io

In [256]:
s = pd.Series(pd.Categorical(['a', 'b', 'b', 'a', 'a', 'd']))

In [258]:
s.cat.categories = ["very good", "good", "bad"]

In [259]:
s

0    very good
1         good
2         good
3    very good
4    very good
5          bad
dtype: category
Categories (3, object): [very good, good, bad]

In [260]:
s = s.cat.set_categories(["very bad", "bad", "medium", "good", "very good"])

In [261]:
df = pd.DataFrame({"cats": s, "vals": [1, 2, 3, 4, 5, 6]})

In [262]:
df

,cats,vals
0,very good,1
1,good,2
2,good,3
3,very good,4
4,very good,5
5,bad,6


In [263]:
csv = io.StringIO()

In [264]:
df.to_csv(csv)

In [266]:
df2 = pd.read_csv(io.StringIO(csv.getvalue()))

In [267]:
df2.dtypes

Unnamed: 0     int64
cats          object
vals           int64
dtype: object

In [268]:
df2["cats"]

0    very good
1         good
2         good
3    very good
4    very good
5          bad
Name: cats, dtype: object

In [270]:
df2["cats"] = df2["cats"].astype("category")

In [271]:
df2["cats"].cat.set_categories(["very bad", "bad", "medium",
    "good", "very good"],
    inplace=True)

In [272]:
df2.dtypes

Unnamed: 0       int64
cats          category
vals             int64
dtype: object

In [273]:
df2["cats"]

0    very good
1         good
2         good
3    very good
4    very good
5          bad
Name: cats, dtype: category
Categories (5, object): [very bad, bad, medium, good, very good]

The same holds for writing to a SQL database with to_sql.

## Missing data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations. See the Missing Data section.

Missing values should not be included in the Categorical’s categories, only in the values. Instead, it is understood that NaN is different, and is always a possibility. When working with the Categorical’s codes, missing values will always have a code of -1.

In [274]:
s = pd.Series(["a", "b", np.nan, "a"], dtype="category")

In [275]:
s

0      a
1      b
2    NaN
3      a
dtype: category
Categories (2, object): [a, b]

In [276]:
 s.cat.codes

0    0
1    1
2   -1
3    0
dtype: int8

Methods for working with missing data, e.g. isna(), fillna(), dropna(), all work normally:

In [277]:
s = pd.Series(["a", "b", np.nan], dtype="category")

In [278]:
s

0      a
1      b
2    NaN
dtype: category
Categories (2, object): [a, b]

In [279]:
pd.isna(s)

0    False
1    False
2     True
dtype: bool

In [280]:
s.fillna("a")

0    a
1    b
2    a
dtype: category
Categories (2, object): [a, b]

## Gotchas

### Memory usage

The memory usage of a Categorical is proportional to the number of categories plus the length of the data. In contrast, an object dtype is a constant times the length of the data.

In [281]:
s = pd.Series(['foo', 'bar'] * 1000)

In [282]:
s.nbytes

16000

In [283]:
s.astype('category').nbytes

2016

> Note: If the number of categories approaches the length of the data, the Categorical will use nearly the same or more memory than an equivalent object dtype representation.

In [284]:
s = pd.Series(['foo%04d' % i for i in range(2000)])

In [285]:
s.nbytes

16000

In [286]:
s.astype('category').nbytes

20000

### Categorical is not a numpy array

Currently, categorical data and the underlying Categorical is implemented as a Python object and not as a low-level NumPy array dtype. This leads to some problems.

NumPy itself doesn’t know about the new dtype:

In [287]:
try:
    np.dtype("category")
except TypeError as e:
    print("TypeError:", str(e))

TypeError: data type "category" not understood


In [288]:
dtype = pd.Categorical(["a"]).dtype

In [289]:
dtype

CategoricalDtype(categories=['a'], ordered=False)

In [290]:
try:
    np.dtype(dtype)
except TypeError as e:
    print("TypeError:", str(e))

TypeError: data type not understood


Dtype comparisons work:

In [291]:
dtype == np.str_

False

In [292]:
np.str_ == dtype

False

To check if a Series contains Categorical data, use hasattr(s, 'cat'):

In [293]:
hasattr(pd.Series(['a'], dtype='category'), 'cat')

True

In [294]:
hasattr(pd.Series(['a']), 'cat')

False

Using NumPy functions on a Series of type category should not work as Categoricals are not numeric data (even in the case that .categories is numeric).

In [295]:
s = pd.Series(pd.Categorical([1, 2, 3, 4]))

In [296]:
try:
    np.sum(s)
except TypeError as e:
    print("TypeError:", str(e))

TypeError: Categorical cannot perform the operation sum


### dtype in apply

Pandas currently does not preserve the dtype in apply functions: If you apply along rows you get a Series of object dtype (same as getting a row -> getting one element will return a basic type) and applying along columns will also convert to object. NaN values are unaffected. You can use fillna to handle missing values before applying a function.

In [297]:
df = pd.DataFrame({"a": [1, 2, 3, 4],
    "b": ["a", "b", "c", "d"],
    "cats": pd.Categorical([1, 2, 3, 2])})

In [298]:
df.apply(lambda row: type(row["cats"]), axis=1)

0    <class 'int'>
1    <class 'int'>
2    <class 'int'>
3    <class 'int'>
dtype: object

In [299]:
df.apply(lambda col: col.dtype, axis=0)

a          int64
b         object
cats    category
dtype: object

### Categorical index

CategoricalIndex is a type of index that is useful for supporting indexing with duplicates. This is a container around a Categorical and allows efficient indexing and storage of an index with a large number of duplicated elements. See the advanced indexing docs for a more detailed explanation.

Setting the index will create a CategoricalIndex:

In [300]:
cats = pd.Categorical([1, 2, 3, 4], categories=[4, 2, 3, 1])

In [301]:
strings = ["a", "b", "c", "d"]

In [303]:
values = [4, 2, 3, 1]

In [304]:
df = pd.DataFrame({"strings": strings, "values": values}, index=cats)

In [306]:
df

,strings,values
1,a,4
2,b,2
3,c,3
4,d,1


In [305]:
df.index

CategoricalIndex([1, 2, 3, 4], categories=[4, 2, 3, 1], ordered=False, dtype='category')

In [307]:
df.sort_index()

,strings,values
4,d,1
2,b,2
3,c,3
1,a,4


### Side effects

Constructing a Series from a Categorical will not copy the input Categorical. This means that changes to the Series will in most cases change the original Categorical:

In [309]:
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])

In [310]:
s = pd.Series(cat, name="cat")

In [314]:
s

0    10
1    10
2     3
3    10
Name: cat, dtype: category
Categories (5, int64): [1, 2, 3, 4, 10]

In [311]:
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [312]:
s.iloc[0:2] = 10

In [313]:
cat

[10, 10, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [315]:
df = pd.DataFrame(s)

In [316]:
df["cat"].cat.categories = [1, 2, 3, 4, 5]

In [317]:
cat

[5, 5, 3, 5]
Categories (5, int64): [1, 2, 3, 4, 5]

Use copy=True to prevent such a behaviour or simply don’t reuse Categoricals:

In [320]:
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])

In [321]:
s = pd.Series(cat, name="cat", copy=True)

In [322]:
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [323]:
s.iloc[0:2] = 10

In [324]:
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

> Note: This also happens in some cases when you supply a NumPy array instead of a Categorical: using an int array (e.g. np.array([1,2,3,4])) will exhibit the same behavior, while using a string array (e.g. np.array(["a","b","c","a"])) will not.